In [109]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pycreackdb/Segmentation/Ground truth/173.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/248.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/94.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/236.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/340.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/48.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/350.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/314.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/227.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/238.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/61.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/222.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/330.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/278.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/303.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/37.png
/kaggle/input/pycreackdb/Segmentation/Ground truth/231.png
/

In [122]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [123]:
# ==================== PATHS ====================
pycrack_path = "/kaggle/input/pycreackdb/Classification"  # PyCrackDB
trib_path    = "/kaggle/input/py-crackdb/TRIB Crack Dataset/3. Resized and augmented images"  # TRIB

# ==================== A) TRAIN/TEST SPLIT ON PYCRACKDB ====================
print("----- Loading PyCrackDB Dataset -----")

train_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_ds = train_gen.flow_from_directory(
    pycrack_path, target_size=(224,224), batch_size=32, shuffle=True, subset='trainig'
)
test_ds = train_gen.flow_from_directory(
    pycrack_path, target_size=(224,224), batch_size=32, shuffle=False, subset='validation'
)


----- Loading PyCrackDB Dataset -----
Found 456 images belonging to 2 classes.
Found 113 images belonging to 2 classes.


In [124]:
# ==================== B) CNN MODEL ====================
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    AveragePooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    AveragePooling2D(pool_size=(2,2)),
    Conv2D(128, (3,3), activation='relu'),
    AveragePooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(train_ds.num_classes, activation='softmax')
])

print("----- CNN Model Summary -----")
model.summary()

----- CNN Model Summary -----


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_21            │ (None, 111, 111, 32)   │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_22            │ (None, 54, 54, 64)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_23            │ (None, 26, 26, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,218 (42.61 MB)

 Trainable params: 11,169,218 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [128]:
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, epochs=20, validation_data=test_ds)


Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.6403 - loss: 0.6319 - val_accuracy: 0.6460 - val_loss: 0.6083
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.6501 - loss: 0.6113 - val_accuracy: 0.6460 - val_loss: 0.5606
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.6272 - loss: 0.5902 - val_accuracy: 0.6637 - val_loss: 0.4858
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.7046 - loss: 0.5390 - val_accuracy: 0.9735 - val_loss: 0.4778
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.7852 - loss: 0.5454 - val_accuracy: 0.8584 - val_loss: 0.5415
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7298 - loss: 0.5743 - val_accuracy: 0.9646 - val_loss: 0.4894
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.7456 - loss: 0.5498 - val_accuracy: 0.9646 - val_loss: 0.4102
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.7520 - loss: 0.5453 - val_accuracy: 0.7168 - val_loss:

In [129]:
loss, acc = model.evaluate(test_ds)
print("\nPyCrackDB Test Accuracy:", acc)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 479ms/step - accuracy: 0.9929 - loss: 0.1991

PyCrackDB Test Accuracy: 0.982300877571106


In [130]:
model.save("best_model.h5")
print("Saved: best_model.h5")


Saved: best_model.h5


In [138]:
 C) GRAD-CAM FUNCTIO
def grad_cam(img, model, layer):
    img_tensor = np.expand_dims(img, axis=0)
    heat_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer).output, model.output])
    with tf.GradientTape() as tape:
        conv_out, preds = heat_model(img_tensor)
        loss = preds[:, tf.argmax(preds[0])]
    grads = tape.gradient(loss, conv_out)[0]
    weights = tf.reduce_mean(grads, axis=(0,1))
    cam = np.maximum(np.sum(weights * conv_out[0], axis=-1), 0)
    return cam / np.max(cam)

In [139]:
# D) BUILD MODEL FOR GRAD-CAM 
model.build((None, 224,224,3))  # Fix for AttributeError
last_conv_layer = [layer.name for layer in model.layers if 'conv2d' in layer.name][-1]
print("Using last Conv2D layer for Grad-CAM:", last_conv_layer)


Using last Conv2D layer for Grad-CAM: conv2d_24


In [135]:

def grad_cam(img, model, layer):
    img_tensor = np.expand_dims(img, axis=0)

    # create heatmap model
    heat_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_out, preds = heat_model(img_tensor)
        class_idx = tf.argmax(preds[0])
        loss = preds[:, class_idx]

    grads = tape.gradient(loss, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))

    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_out[0]), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (np.max(heatmap) + 1e-9)
    return heatmap


In [ ]:
 F) GENERALIZABILITY TEST: TRIB DATASET
print("----- Generalizability Test: TRIB Dataset -----")

train2 = ImageDataGenerator(rescale=1./255).flow_from_directory(
    trib_path, target_size=(256,256), batch_size=32, shuffle=True
)
test2  = ImageDataGenerator(rescale=1./255).flow_from_directory(
    trib_path, target_size=(256,256), batch_size=32, shuffle=False
)

model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train2, epochs=3)

loss2, acc2 = model.evaluate(test2)
print("TRIB Test Accuracy:", acc2)
print("PyCrackDB Test Accuracy:", acc)
print("Performance difference (PyCrackDB - TRIB):", acc-acc2)

model.save("best_model_TRIB.h5")
print("Saved: best_model_TRIB.h5")